# Zmienne kategoryczne w sklearn + szybki ogląd danych

Sci-kit learn w obecnej wersji wymaga, aby ramka dostarczana do metody `.fit` zawierała *tylko* wartości  numeryczne (kolumny boolowe zostaną zrzutowane do wartości `[1, 0]`). Oczekuje się więc od użytkownika, że cechy zawierające typ `string`, czy `datetime` zostaną zakodowane tak, aby klasyfikator mógł wyekstrahować informację zawartą w danych. Ale **UWAGA:** nie możemy z góry założyć, że jeśli kolumna zawiera wartość numeryczną (w szczególności: `int`), to że automatycznie nie jest nominalna.

Zaczniemy od przejrzenia wartości w poszczególnych kolumnach naszych danych treningowych. Chcemy znaleźć cechy nominalne (kategoryczne), które następnie pogrupujemy ze względu na typ, liczbę unikatowych wartości, oraz inne kryteria.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
x_and_y_train = pd.concat([pd.read_csv('data/xtrain_1.csv'),
                           pd.read_csv('data/xtrain_2.csv'),
                           pd.read_csv('data/y_train.csv', header=None, names=['y'])],
                          axis=1)

x_and_y_train['received_at'] = pd.to_datetime(x_and_y_train['received_at'])

x_and_y_train = x_and_y_train.sort_values('received_at')

ytrain = x_and_y_train.pop('y')
xtrain = x_and_y_train

In [ ]:
for colname in sorted(xtrain.columns):
    not_nones = xtrain[colname][pd.notnull(xtrain[colname])]
    has_complex_types = any(not_nones.apply(lambda x: isinstance(x, (dict, list))))
    is_not_Series = not isinstance(not_nones, pd.Series)
    if has_complex_types or is_not_Series:
        continue
    unique_values = not_nones.unique()
    print('{} ({}): {}'.format(colname, len(unique_values), unique_values[:10]))

In [ ]:
low_card_cat_colnames = [
                    'agent_type', 'bin_country', 'browser', 'browser_string_mismatch',
                    'card_brand', 'card_level', 'card_type', 'cc_bin_number_brand', 'cc_bin_number_type',
                    'cc_number_hash_assert_history', 'cc_number_hash_result', 'input_ip_assert_history',
                    'input_ip_geo', 'input_ip_result', 'js_browser', 'js_fonts_number', 'js_os', 'navigatorplatform',
                    'os', 'remote_desktop', 'request_result', 'review_status', 'x10047', 'x12050', 'x16453', 
                    'x21202', 'x21215', 'x21877', 'x26431', 'x32520', 'x39150', 'x41009', 'x5170', 'x5193',
                    'x5290', 'x5342', 'x55586', 'x66439', 'x70617', 'x72496', 'x73629', 'x80025', 'x80385',
                    'x80911', 'x83032', 'x83336', 'x91746', 'x92645', 'x96255', 
                   ]
                    
                    
high_card_cat_colnames = [
                    'bin', 'browser_string_hash', 'cc_bin_number', 'cc_bin_number_category', 'cc_bin_number_geo',
                    'cc_bin_number_org', 'country_code', 'dns_ip_city', 'dns_ip_isp', 'dns_ip_organization',
                    'dns_ip_region', 'input_ip_city', 'js_browser_string_hash', 'js_fonts_hash', 'language',
                    'mime_type_hash', 'name', 'user_ip_country', 'x12964', 'x18591', 'x2801', 'x30901', 'x33709',
                    'x46567', 'x46591', 'x52094', 'x55037', 'x61305', 'x63143', 'x66015', 'x76585', 'x82598',
                    'x87611', 'x92166', 'x92221', 'x94347', 
                    ]
                    
parsable_cat_colnames = [
                    'browser_string', 'headers_accept_encoding', 'headers_accept_language', 'headers_user_agent',
                    'input_ip_attributes', 'js_browser_string', 'navigatorappversion', 'useragent', 'x31255',
                    'x50315', 
                    ]
                    
degree_cat_colnames = ['risk_rating']
                    
castable_to_numeric_cat_colnames = [
                    'x23739', 'x3314', 'x42317', 'x43300', 'x44170', 'x48420', 'x51274', 'x54084',
                    'x5600', 'x59752', 'x60781', 'x87918', 'x9486', 'x97674', 'x98509',
                    'application_id'
                    ]

# Modele budowane na zmiennych kategorycznych

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeClassifier
from sklearn.preprocessing import Imputer

from xgboost import XGBClassifier

import matplotlib.pyplot as plt
%matplotlib inline

import category_encoders as ce

from hash_transformers import SimplisticHasher

## Modele budowane na `low_card_cat_colnames`

In [ ]:
DF_cat = xtrain[low_card_cat_colnames]
DF_cat.head()

**WAŻNA UWAGA:** zwróćcie uwagę, że metoda podziału na treningowy/walidacyjny używana poniżej (`TimeSeriesSplit`) **zakłada**, że ramka jest posortowana wg odpowiedniej kolumny czasowej (tutaj: `received_at`).

In [ ]:
cat_colnames = DF_cat.columns.tolist()

pipe = Pipeline([
#     ('hashing_trick', ce.HashingEncoder(cols=cat_colnames, n_components=10*len(cat_colnames))),
    ('simplistic_hash', SimplisticHasher(cols=cat_colnames)),
    
    ('imputer', Imputer(missing_values=pd.np.nan, strategy='most_frequent', axis=0)),    
#     ('xgb', XGBClassifier()),
    ('log_reg', LogisticRegression()),
#     ('lin_reg', LinearRegression()),
#     ('ridge', RidgeClassifier()),
])


scores = cross_val_score(pipe,
                         DF_cat,
                         ytrain,
                         scoring='roc_auc',
                         cv=TimeSeriesSplit(6))

plt.plot(scores)

## Modele budowane na `[low_card_cat_colnames, high_card_cat_colnames]`

In [ ]:
DF_cat = xtrain[low_card_cat_colnames+high_card_cat_colnames]

cat_colnames = DF_cat.columns.tolist()

pipe = Pipeline([
    ('hashing_trick', ce.HashingEncoder(cols=cat_colnames, n_components=10*len(cat_colnames))),
#     ('simplistic_hash', SimplisticHasher(cols=cat_colnames)),
    
#     ('imputer', Imputer(missing_values=pd.np.nan, strategy='most_frequent', axis=0)),    
    ('xgb', XGBClassifier()),
#     ('log_reg', LogisticRegression()),
#     ('lin_reg', LinearRegression()),
#     ('ridge', RidgeClassifier()),
])


scores = cross_val_score(pipe,
                         DF_cat,
                         ytrain,
                         scoring='roc_auc',
                         cv=TimeSeriesSplit(6))

plt.plot(scores)

# Kolumny zawierające słowniki

In [ ]:
def undict_one_col(df, colname):
    column = df.pop(colname)
    values = [dict() if x is None else x for x in column]
    unpacked_columns = pd.DataFrame.from_dict(values)
    unpacked_columns = unpacked_columns.add_prefix(colname + '_')
    return pd.concat([df, unpacked_columns], axis=1)

def undict_cols(df, col_names):
    for col_name in col_names:
        df = undict_one_col(df, col_name)    
    return df


dict_counts = xtrain.apply(lambda col: col.apply(lambda x: isinstance(x, dict))).sum()
dict_colnames = dict_counts[dict_counts>0].index.tolist()

xtrain_undicted = undict_cols(xtrain[dict_colnames], dict_colnames)
xtrain_undicted_numeric = xtrain_undicted.select_dtypes(include=[np.number])

## Model zbudowany na zmiennych kategorycznych + numerycznych ze słowników

In [ ]:
DF_cat_and_undict = pd.concat([DF_cat, xtrain_undicted_numeric], axis=1)
cat_colnames = DF_cat_and_undict.select_dtypes(exclude=[np.number]).columns.tolist()

DF_cat_and_undict.head()

In [ ]:
scores = cross_val_score(pipe,
                         DF_cat_and_undict,
                         ytrain,
                         scoring='roc_auc',
                         cv=TimeSeriesSplit(6))

plt.plot(scores)

# Odnośniki

#### One Hot Encoding i algorytmy drzewiaste
Post: http://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/ omawia przykład, w którym kodowanie zmiennych nominalnych przy użyciu OHE porównywane jest z kodowaniem wykorzystywanym domyślnie w bibliotece H2O (http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/nbins_cats.html). Dodatkowo: paragraf *4.6 Treatment of factors* w opisie GBMów w H2O https://h2o-release.s3.amazonaws.com/h2o/master/3153/docs-website/h2o-docs/booklets/GBM_Vignette.pdf



#### Wykład Owena Zhanga
https://www.youtube.com/watch?v=LgLcfZjNF44, w którym mowa o: GBMach, kodowaniu zmiennych nominalnych, licznych "sztuczkach" przydatnych w konkursach kaggle'owych. 



#### GBMy w ramach Distributed Machine Learning Toolkit
https://github.com/Microsoft/LightGBM; w swoim benchmarku autorzy twierdzą, LightGBM daje w niektórych przypadkach lepsze wyniki, niż XGBoost (https://github.com/Microsoft/LightGBM/wiki/Experiments#comparison-experiment).



#### Pull request na repo sklearnowym odn. wprowadzenia natywnego wsparcia dla zmiennych kategorycznych
https://github.com/scikit-learn/scikit-learn/pull/4899



#### Repo paczki `category_encoders`
https://github.com/scikit-learn-contrib/categorical-encoding

# Praca domowa
(do wykonania grupowo)

0. Zbierzcie powielone kawałki kodu z tego notatnika i opakujcie je w funkcje, które będą w plikach `.py` (tak, żeby można je było importować);

1. Przejrzyjcie pozostałe typy zmiennych kategorycznych (`parsable_cat_colnames, degree_cat_colnames, castable_to_numeric_cat_colnames`) i wybierzcie te, które chcecie jeszcze dodać do danych treningowych;

2. Rozpakujcie kolumny zawierające wartości typu `list` (polecam szczególnie kolumnę `'flash_fonts'`);

3. Wypróbujcie inne klasyfikatory (`sklearn.ensemble.AdaBoostClassifier`, `sklearn.svm.SVC`);


# Przypomnienie

Jeśli chcecie popracować na AWSie w domu, ustalcie jaki termin by Wam najbardziej pasował i dajcie znać na maila: **maciej.dziubinski@daftcode.pl**